In [1]:
# Install necessary packages
!pip install tensorflow matplotlib

# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt


In [2]:
def build_model(input_shape, style_layers, content_layers):
    # Define the VGG19 model pretrained on ImageNet
    base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    # Get the intermediate layers for style and content representations
    style_outputs = [base_model.get_layer(name).output for name in style_layers]
    content_outputs = [base_model.get_layer(name).output for name in content_layers]

    model_outputs = style_outputs + content_outputs

    # Build and compile the model
    model = models.Model(inputs=base_model.input, outputs=model_outputs)
    model.compile(optimizer='adam', loss='mse')  # Adjust loss function based on your needs

    return model


In [3]:
def style_loss(style, combination):
    # Implement the Gram matrix comparison for style loss
    return K.mean(K.square(style - combination))

def content_loss(content, combination):
    # Use mean squared error for content loss
    return K.mean(K.square(content - combination))


In [13]:
def train_model(model, content_image, style_image, epochs=10, content_weight=1e3, style_weight=1e-2):
    # Preprocess the input images
    content_image = tf.keras.applications.vgg19.preprocess_input(content_image)
    style_image = tf.keras.applications.vgg19.preprocess_input(style_image)

    # Extract features for content and style images
    content_features = model(content_image)[-1]  # Assuming the last layer for content
    style_features = model(style_image)[:len(style_layers)]  # Select the specified layers

    # Create a tensor to hold the style and content targets
    targets = style_features + [content_features]

    # Create a weighted combination of the style and content losses
    weights = [style_weight] * len(style_features) + [content_weight]

    # Define the total loss as a combination of style and content losses
    def total_loss(outputs, targets, weights):
        total_loss = 0
        for i in range(len(outputs)):
            total_loss += weights[i] * content_loss(targets[i], outputs[i])
        return total_loss

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)  # Adjust the learning rate as needed

    # Training loop
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            outputs = model(content_image)
            loss = total_loss(outputs, targets, weights)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        # Print the loss every few epochs
        if epoch % 5 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")


In [18]:
def style_transfer(model, content_image, style_image):
    # Preprocess the input images
    content_image = tf.keras.applications.vgg19.preprocess_input(content_image)
    style_image = tf.keras.applications.vgg19.preprocess_input(style_image)

    # Extract features for the content image
    content_features = model(content_image)[-1]

    # Extract features for the style image
    style_features = model(style_image)[:len(style_layers)]

    # Create a tensor to hold the style and content targets
    targets = style_features + [content_features]

    # Generate a random image as the initial stylized image
    generated_image = tf.random.uniform(content_image.shape, minval=0, maxval=255)

    # Define the total loss as a combination of style and content losses
    def total_loss(outputs, targets, weights):
        total_loss = 0
        for i in range(len(outputs)):
            total_loss += weights[i] * content_loss(targets[i], outputs[i])
        return total_loss

    # Optimization loop
    for _ in range(epochs):
        with tf.GradientTape() as tape:
            outputs = model(generated_image)
            loss = total_loss(outputs, targets, weights)

        gradients = tape.gradient(loss, generated_image)
        optimizer.apply_gradients([(gradients, generated_image)])

    return generated_image.numpy()


In [21]:
def style_transfer(model, content_image, style_image, epochs=10):
    # Preprocess the input images
    content_image = tf.keras.applications.vgg19.preprocess_input(content_image)
    style_image = tf.keras.applications.vgg19.preprocess_input(style_image)

    # Extract features for the content image
    content_features = model(content_image)[-1]

    # Extract features for the style image
    style_features = model(style_image)[:len(style_layers)]

    # Create a tensor to hold the style and content targets
    targets = style_features + [content_features]

    # Generate a random image as the initial stylized image
    generated_image = tf.random.uniform(content_image.shape, minval=0, maxval=255)

    # Define the total loss as a combination of style and content losses
    def total_loss(outputs, targets, weights):
        total_loss = 0
        for i in range(len(outputs)):
            total_loss += weights[i] * tf.reduce_mean(tf.square(targets[i] - outputs[i]))
        return total_loss

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    # Optimization loop
    for _ in range(epochs):
        with tf.GradientTape() as tape:
            outputs = model(generated_image)
            loss = total_loss(outputs, targets, weights)

        gradients = tape.gradient(loss, generated_image)
        optimizer.apply_gradients([(gradients, generated_image)])

    return generated_image.numpy()


In [23]:
def style_transfer(model, content_image, style_image, epochs=10, weights=None):
    # Preprocess the input images
    content_image = tf.keras.applications.vgg19.preprocess_input(content_image)
    style_image = tf.keras.applications.vgg19.preprocess_input(style_image)

    # Extract features for the content image
    content_features = model(content_image)[-1]

    # Extract features for the style image
    style_features = model(style_image)[:len(style_layers)]

    # Create a tensor to hold the style and content targets
    targets = style_features + [content_features]

    # Generate a random image as the initial stylized image
    generated_image = tf.random.uniform(content_image.shape, minval=0, maxval=255)

    # Define the total loss as a combination of style and content losses
    def total_loss(outputs, targets, weights):
        total_loss = 0
        for i in range(len(outputs)):
            total_loss += weights[i] * tf.reduce_mean(tf.square(targets[i] - outputs[i]))
        return total_loss

    # Check if weights are not provided, use default
    if weights is None:
        weights = [1e-2] * len(style_features) + [1e3]

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    # Optimization loop
    for _ in range(epochs):
        with tf.GradientTape() as tape:
            outputs = model(generated_image)
            loss = total_loss(outputs, targets, weights)

        gradients = tape.gradient(loss, generated_image)
        optimizer.apply_gradients([(gradients, generated_image)])

    return generated_image.numpy()


In [24]:
# Load your content and style images
content_image = tf.keras.preprocessing.image.load_img('1_a.jpg', target_size=(256, 256))
style_image = tf.keras.preprocessing.image.load_img('r1.jpg', target_size=(256, 256))

# Convert images to NumPy arrays
content_image = tf.keras.preprocessing.image.img_to_array(content_image)
style_image = tf.keras.preprocessing.image.img_to_array(style_image)

# Add batch dimensions
content_image = tf.expand_dims(content_image, axis=0)
style_image = tf.expand_dims(style_image, axis=0)

# Set up the model
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
content_layers = ['block5_conv2']
model = build_model((256, 256, 3), style_layers, content_layers)

# Train the model
train_model(model, content_image, style_image, epochs=10)

# Perform style transfer
stylized_image = style_transfer(model, content_image, style_image)

# Display the results
plt.subplot(1, 3, 1)
plt.imshow(content_image[0] / 255.0)
plt.title('Content Image')

plt.subplot(1, 3, 2)
plt.imshow(style_image[0] / 255.0)
plt.title('Style Image')

plt.subplot(1, 3, 3)
plt.imshow(stylized_image[0] / 255.0)
plt.title('Stylized Image')

plt.show()


Epoch 1, Loss: 22111.57421875
Epoch 6, Loss: 22111.57421875


AttributeError: Tensor.name is undefined when eager execution is enabled.